In [1]:
##get data
import numpy as np
import sys
sys.path
sys.path.append('../models')
import pickle
import math
import tensorflow as tf
import gpflow

In [2]:
##set grid
u1 = np.arange(.1,1.1,.1)
u2 = np.arange(.1,1.1,.1)
rho = np.arange(.1,1.1,.2)

In [3]:
v1 = np.arange(1,10,1)
v2 = np.arange(1,10,1)

In [4]:
noise = np.arange(.1,1.1,.1)

In [5]:
with open('../../Downloads/processed_data_for_gridsearch_GP.pkl','rb') as f:
    data = pickle.load(f)

In [6]:
X=data[0]
y=data[1]

In [7]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))


In [8]:
import CustomKernelStatic

In [9]:
def get_likelihood(u1,u2,s1,s2,rho,noise):
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(10,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[i for i in range(18)],\
                  psi_indices=[0,7],user_day_index=19,user_index=18,num_data_points=len(X),initial_u1=u1,initial_u2=u2,\
                           initial_s1=s1,initial_s2=s2,initial_rho=rho,initial_noise=noise)
  
    m = gpflow.models.GPR(X,y, kern=k)
    m.initialize(session=sess)
    try:
        l =m.compute_log_likelihood(X)
    except:
        l=-100
    return l
    ##get likelihood #return likelihood

In [12]:
def run():
    to_save = {}
    u1 = np.arange(.1,1.1,.1)
    u2 = np.arange(.1,1.1,.1)
    rhos = np.arange(.1,1.1,.2)
    v1 = np.arange(1,10,1)
    v2 = np.arange(1,10,1)
    noise = np.arange(.1,1.1,.1)
    
    with open('../../Downloads/processed_data_for_gridsearch_GP.pkl','rb') as f:
        data = pickle.load(f)
    
    X=data[0]
    y=data[1]
    
    rdayone = [x[19] for x in X]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    users = np.array([[float(X[i][18]==X[j][18]) for j in range(len(X))] for i in range(len(X))])
    
    for uone in u1:
        for utwo in u2:
            for rho in rhos:
                for s1 in v1:
                    for s2 in v2:
                        for n in noise:
                            m=get_likelihood(uone,uone,s1,s2,rho,n)
                            print(m)
                            key = '-'.join([str(i) for i in[uone,utwo,rho,s1,s2,n]])
                            to_save[key]=m
    return to_save

In [13]:
 run()

ValueError: Shapes must be equal rank, but are 0 and 1
	From merging shape 0 with other shapes. for 'GPR-1b57a717-10/likelihood_1/stack_1' (op: 'Pack') with input shapes: [], [6286].

In [11]:
'-'.join([str(i) for i in [u1[0],u2[0],rho[0],v1[0],v2[0],noise[0]]])

'0.1-0.1-0.1-1-1-0.1'

In [36]:
tod,dow,loc,weather,pre
#m.objective

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,1.0
